<a href="https://colab.research.google.com/github/kilosonc/kaggle/blob/master/dog_and_cat/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
from google.colab import userdata
with open("~/.kaggle/kaggle.json", "w") as f:
  f.write(userdata.get('kaggle'))
!kaggle competitions download -c dog-vs-cat-classification
!yes | unzip dog-vs-cat-classification.zip -d ./dataset > /dev/null
!rm -f dog-vs-cat-classification.zip

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: ./dataset/train/train/dogs/dog.5499.jpg  
  inflating: ./dataset/train/train/dogs/dog.55.jpg  
  inflating: ./dataset/train/train/dogs/dog.550.jpg  
  inflating: ./dataset/train/train/dogs/dog.5500.jpg  
  inflating: ./dataset/train/train/dogs/dog.5501.jpg  
  inflating: ./dataset/train/train/dogs/dog.5502.jpg  
  inflating: ./dataset/train/train/dogs/dog.5503.jpg  
  inflating: ./dataset/train/train/dogs/dog.5504.jpg  
  inflating: ./dataset/train/train/dogs/dog.5505.jpg  
  inflating: ./dataset/train/train/dogs/dog.5506.jpg  
  inflating: ./dataset/train/train/dogs/dog.5507.jpg  
  inflating: ./dataset/train/train/dogs/dog.5508.jpg  
  inflating: ./dataset/train/train/dogs/dog.5509.jpg  
  inflating: ./dataset/train/train/dogs/dog.551.jpg  
  inflating: ./dataset/train/train/dogs/dog.5510.jpg  
  inflating: ./dataset/train/train/dogs/dog.5511.jpg  
  inflating: ./dataset/train/train/dogs/dog.5512.jpg  
  inflating: ./dataset/train/train/dogs/do

In [2]:
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Sequential(
            nn.Conv2d(
                3, 512, 5
            ),
            nn.ReLU(),
            nn.MaxPool2d((10,10))
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(512, 1024, 5),
            nn.ReLU(),
            nn.MaxPool2d((10,10))
        )
        self.output = nn.Sequential(
            # nn.Linear(3810304, 3721),
            # nn.ReLU(),
            nn.Linear(4096, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 200),
            nn.ReLU(),
            nn.Linear(200, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
            nn.ReLU(),
            nn.Linear(10, 2),
        )

    def forward(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        return self.output(x)

In [4]:
# 检查 MPS 可用性
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Apple GPU
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")  # 回退到 CPU
print(f"Using device: {device}")

Using device: cuda


In [5]:
cnn = Net()
cnn.to(device=device)
print(cnn)

Net(
  (conv1): Sequential(
    (0): Conv2d(3, 512, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(10, 10), stride=(10, 10), padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=(10, 10), stride=(10, 10), padding=0, dilation=1, ceil_mode=False)
  )
  (output): Sequential(
    (0): Linear(in_features=4096, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=200, bias=True)
    (5): ReLU()
    (6): Linear(in_features=200, out_features=50, bias=True)
    (7): ReLU()
    (8): Linear(in_features=50, out_features=10, bias=True)
    (9): ReLU()
    (10): Linear(in_features=10, out_features=2, bias=True)
  )
)


In [6]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os

class DogsAndCatsDataset(Dataset):
    def __init__(self):
        data = []
        for (path, _, files) in os.walk("./dataset/train"):
            for f in files:
                index = int(f.split(".")[1])
                file_path = f"{path}/{f}"
                if path.endswith("dogs"):
                    data.append([index, file_path, "dog"])
                else:
                    data.append([index, file_path, "cat"])
        self.train_set = pd.DataFrame(data, columns=["id", "path", "type"])
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize(0.5, 0.5)
        ])

    def __len__(self):
        return len(self.train_set)

    def __getitem__(self, index):
        item = self.train_set.loc[index, ["path","type"]]
        path = item.loc["path"]
        label = 1 if item.loc["type"] == "dog" else 0
        image = Image.open(path).convert("RGB")
        return self.transform(image), label

data = DogsAndCatsDataset()
img, _ = data[1]
print(img.size())

torch.Size([3, 256, 256])


In [7]:
from torch.utils.data import DataLoader

loader = DataLoader(dataset=data,batch_size=10,shuffle=True)

In [ ]:
from tqdm import tqdm
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

for epoch in range(10):
    for imgs , labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        output = cnn(imgs.view(-1, 3, 256, 256).to(device))
        loss = loss_func(output, labels)
        loss.backward()
        optimizer.step()



 53%|█████▎    | 1337/2500 [03:36<03:08,  6.17it/s]